In [1]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *
import folium
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
plotly.tools.set_credentials_file(username='silkeh', api_key='GwBAmaelgKfgkrmuL691')

In [6]:
#IMDB
#title.basics
#tconst, titleType, primaryTitle, originalTitle, isAdult, startYear, endYear, runtimeMinutes, genres
titleBasics_data = (sc.textFile('D:/Workspace/_DataMining/DataSets/IMDB/titleBasics.tsv'))
titleBasics_rdd = titleBasics_data.filter(lambda l: 'primaryTitle' not in l)
                   
#create schema for df
#tbFields = titleBasicsFields
tbFields = []
tbFields.append(StructField('tconst', StringType(), True))
tbFields.append(StructField('titleType', StringType(), True))
tbFields.append(StructField('primaryTitle', StringType(), True))
tbFields.append(StructField('originalTitle', StringType(), True))
tbFields.append(StructField('isAdult', BooleanType(), True))
tbFields.append(StructField('startYear', IntegerType(), True))
tbFields.append(StructField('endYear', IntegerType(), True))
tbFields.append(StructField('runtimeMinutes', IntegerType(), True))
tbFields.append(StructField('genres', StringType(), True))                           

tbSchema = StructType(tbFields)

#create df
titleBasics_df= sqlContext.createDataFrame(titleBasics_rdd, tbSchema)

titleBasics_df.show(n=2)

#title.akas
#titleId, ordering, title, region, language, types, attributes, isOriginalTitle
titleAkas_data = (sc.textFile('D:/Workspace/_DataMining/DataSets/IMDB/titleAkas.tsv'))
titleAkas_rdd = titleAkas_data.filter(lambda l: 'ordering' not in l)

taFields = []
taFields.append(StructField('titleId', StringType(), True))
taFields.append(StructField('ordering', IntegerType(), True))
taFields.append(StructField('title', StringType(), True))
taFields.append(StructField('region', StringType(), True))
taFields.append(StructField('language', StringType(), True))
taFields.append(StructField('types', StringType(), True))
taFields.append(StructField('attributes', StringType(), True))
taFields.append(StructField('isOriginalTitle', BooleanType(), True))

taSchema = StructType(taFields)

titleAkas_df = sqlContext.creatDataFrame(titleAkas_rdd, taSchema)

titleAkas_df.show(n=2)

Py4JJavaError: An error occurred while calling o196.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3.0 (TID 3, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "D:\DevTools\spark-2.3.1-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 230, in main
  File "D:\DevTools\spark-2.3.1-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 225, in process
  File "D:\DevTools\spark-2.3.1-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\serializers.py", line 372, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "D:\DevTools\spark-2.3.1-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\util.py", line 55, in wrapper
    return f(*args, **kwargs)
  File "D:\DevTools\spark-2.3.1-bin-hadoop2.7\python\pyspark\sql\session.py", line 673, in prepare
    verify_func(obj)
  File "D:\DevTools\spar